In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-performance-in-exams/StudentsPerformance.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report
from sklearn.tree import plot_tree

# Suppress specific warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Load the dataset
data_path = '/kaggle/input/students-performance-in-exams/StudentsPerformance.csv'
df = pd.read_csv(data_path)

# Display basic information about the dataset
print("Dataset shape:", df.shape)
print("\nFirst 5 rows of the dataset:")
print(df.head())
print("\nDataset info:")
print(df.info())
print("\nDescriptive statistics:")
print(df.describe())
print("\nMissing values:")
print(df.isnull().sum())

# Exploratory Data Analysis - Updated to avoid warnings
plt.figure(figsize=(15, 10))

# Distribution of scores
plt.subplot(2, 3, 1)
sns.histplot(df['math score'], kde=True)
plt.title('Math Score Distribution')

plt.subplot(2, 3, 2)
sns.histplot(df['reading score'], kde=True)
plt.title('Reading Score Distribution')

plt.subplot(2, 3, 3)
sns.histplot(df['writing score'], kde=True)
plt.title('Writing Score Distribution')

# Gender distribution
plt.subplot(2, 3, 4)
sns.countplot(x='gender', data=df)
plt.title('Gender Distribution')

# Parent education level
plt.subplot(2, 3, 5)
plt.title('Parental Education Level')
education_counts = df['parental level of education'].value_counts()
sns.barplot(x=education_counts.values, y=education_counts.index)

# Test preparation course
plt.subplot(2, 3, 6)
sns.countplot(x='test preparation course', data=df)
plt.title('Test Preparation Course')

plt.tight_layout()
plt.savefig('eda_plots.png')
plt.close()

# Create a target variable (pass/fail)
# Calculate the average score across all subjects
df['average_score'] = (df['math score'] + df['reading score'] + df['writing score']) / 3

# Define pass threshold (e.g., average score >= 70 is a pass)
threshold = 70
df['passed'] = (df['average_score'] >= threshold).astype(int)

# Display pass/fail distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='passed', data=df)
plt.title(f'Pass/Fail Distribution (Threshold: {threshold})')
plt.xticks([0, 1], ['Failed', 'Passed'])
plt.savefig('pass_fail_distribution.png')
plt.close()

print(f"\nPass/Fail Distribution (Threshold: {threshold}):")
print(df['passed'].value_counts())
print(f"Pass rate: {df['passed'].mean()*100:.2f}%")

# Prepare features and target
X = df.drop(['math score', 'reading score', 'writing score', 'average_score', 'passed'], axis=1)
y = df['passed']

# Define categorical features
categorical_features = ['gender', 'race/ethnicity', 'parental level of education', 
                        'lunch', 'test preparation course']

# Preprocessing for categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ],
    remainder='passthrough'
)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Build and train the models
# 1. Logistic Regression
log_reg_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

log_reg_pipeline.fit(X_train, y_train)
y_pred_log = log_reg_pipeline.predict(X_test)

# 2. Decision Tree
dt_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

dt_pipeline.fit(X_train, y_train)
y_pred_dt = dt_pipeline.predict(X_test)

# Evaluate the models
print("\n--- Logistic Regression Results ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_log):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_log):.4f}")
print("Confusion Matrix:")
cm_log = confusion_matrix(y_test, y_pred_log)
print(cm_log)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_log))

print("\n--- Decision Tree Results ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_dt):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_dt):.4f}")
print("Confusion Matrix:")
cm_dt = confusion_matrix(y_test, y_pred_dt)
print(cm_dt)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_dt))

# Visualize confusion matrices
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.heatmap(cm_log, annot=True, fmt='d', cmap='Blues')
plt.title('Logistic Regression Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(1, 2, 2)
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Blues')
plt.title('Decision Tree Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.savefig('confusion_matrices.png')
plt.close()

# Get preprocessed feature names
preprocessed_X_train = preprocessor.fit_transform(X_train)
feature_names = preprocessor.get_feature_names_out()

# Feature importance analysis
# Logistic Regression feature importance
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(preprocessed_X_train, y_train)

# Get feature importance from logistic regression
coefficients = log_reg.coef_[0]
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': np.abs(coefficients)
})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance.head(10))
plt.title('Top 10 Feature Importance (Logistic Regression)')
plt.tight_layout()
plt.savefig('feature_importance_log_reg.png')
plt.close()

# Decision Tree feature importance
dt = DecisionTreeClassifier(random_state=42)
dt.fit(preprocessed_X_train, y_train)

# Get feature importance from decision tree
dt_feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': dt.feature_importances_
})
dt_feature_importance = dt_feature_importance.sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=dt_feature_importance.head(10))
plt.title('Top 10 Feature Importance (Decision Tree)')
plt.tight_layout()
plt.savefig('feature_importance_dt.png')
plt.close()

# Visualize the Decision Tree (limited depth for visibility)
plt.figure(figsize=(20, 10))
dt_small = DecisionTreeClassifier(max_depth=3, random_state=42)
dt_small.fit(preprocessed_X_train, y_train)
plot_tree(dt_small, feature_names=feature_names,
         class_names=['Fail', 'Pass'], filled=True, rounded=True)
plt.title('Decision Tree Visualization (Limited Depth)')
plt.savefig('decision_tree_viz.png')
plt.close()

# Additional analysis: Feature correlation with performance
print("\nFeature Impact on Performance:")
for feature in categorical_features:
    print(f"\n{feature.upper()} IMPACT:")
    impact = df.groupby(feature)['average_score'].agg(['mean', 'count']).sort_values('mean', ascending=False)
    impact['percent_of_total'] = impact['count'] / impact['count'].sum() * 100
    print(impact)

# Compare the models
print("\n--- Model Comparison ---")
print(f"Logistic Regression Accuracy: {accuracy_score(y_test, y_pred_log):.4f}")
print(f"Decision Tree Accuracy: {accuracy_score(y_test, y_pred_dt):.4f}")
print(f"Logistic Regression Precision: {precision_score(y_test, y_pred_log):.4f}")
print(f"Decision Tree Precision: {precision_score(y_test, y_pred_dt):.4f}")

# Generate recommendations based on findings
print("\n--- RECOMMENDATIONS ---")
print("Based on the feature importance analysis:")
print("1. The most influential factors for student success are:")
for feature, importance in zip(dt_feature_importance['Feature'].head(3), dt_feature_importance['Importance'].head(3)):
    print(f"   - {feature}: {importance:.4f}")
print("\n2. To improve student pass rates, focus on:")
print("   - Proper test preparation")
print("   - Support for students with specific demographic characteristics that correlate with lower performance")
print("   - Addressing lunch program effectiveness as it correlates with performance")

# Threshold analysis - seeing how different thresholds affect pass rates
print("\n--- THRESHOLD ANALYSIS ---")
thresholds = [60, 65, 70, 75, 80]
for thresh in thresholds:
    pass_rate = (df['average_score'] >= thresh).mean() * 100
    print(f"Threshold {thresh}: Pass Rate = {pass_rate:.2f}%")

Dataset shape: (1000, 8)

First 5 rows of the dataset:
   gender race/ethnicity parental level of education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test preparation course  math score  reading score  writing score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75  

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (to